# AlphaScrabble: AlphaZero-style Scrabble Engine

This notebook demonstrates a complete AlphaZero-style Scrabble engine with:
- Monte Carlo Tree Search (MCTS) with neural network guidance
- GADDAG/DAWG lexicon for move generation
- Self-play training pipeline
- Interactive gameplay

**Ready to run in Google Colab Pro with GPU support!**


## 1. Setup and Installation

Install all required dependencies and setup the environment.


In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y qtbase5-dev libqt5core5a build-essential cmake ninja-build

# Install Python dependencies
!pip install -U pip wheel cmake ninja pybind11 pytest tensorboard pandas pyarrow rich tqdm click

# Install PyTorch with CUDA support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("✅ Dependencies installed successfully!")


In [ ]:
# Clone and install AlphaScrabble
!git clone https://github.com/alphascrabble/alphascrabble.git
!cd alphascrabble && pip install -e .

print("✅ AlphaScrabble installed successfully!")


## 2. Lexicon Setup

Download and compile the English Scrabble lexicon from ENABLE1 word list.


In [ ]:
import os
import requests
from pathlib import Path

# Create lexicon cache directory
lexica_dir = Path("lexica_cache")
lexica_dir.mkdir(exist_ok=True)

# Download ENABLE1 word list
enable1_url = "https://norvig.com/ngrams/enable1.txt"
enable1_path = lexica_dir / "enable1.txt"

if not enable1_path.exists():
    print("📥 Downloading ENABLE1 word list...")
    response = requests.get(enable1_url)
    with open(enable1_path, 'w') as f:
        f.write(response.text)
    print(f"✅ Downloaded {enable1_path}")
else:
    print(f"✅ ENABLE1 already exists: {enable1_path}")

# Show first few words
with open(enable1_path, 'r') as f:
    words = f.read().strip().split('\n')[:10]
    print(f"📝 First 10 words: {', '.join(words)}")
    print(f"📊 Total words: {len(f.read().strip().split('\n'))}")


## 3. Neural Network Demo

Create and test the neural network architecture.


In [ ]:
import torch
import numpy as np
import sys
sys.path.append('alphascrabble')

from alphascrabble.nn.model import AlphaScrabbleNet
from alphascrabble.engine.features import FeatureExtractor

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create neural network
print("🧠 Creating neural network...")
model = AlphaScrabbleNet().to(device)
print(f"✅ Model created with {sum(p.numel() for p in model.parameters())} parameters")

# Test forward pass
print("🧪 Testing forward pass...")
feature_extractor = FeatureExtractor()

# Create dummy features
board_features = np.random.rand(32, 15, 15).astype(np.float32)
rack_features = np.random.rand(27).astype(np.float32)
move_features = np.random.rand(5, 64).astype(np.float32)

# Convert to tensors
board_tensor = torch.FloatTensor(board_features).unsqueeze(0).to(device)
rack_tensor = torch.FloatTensor(rack_features).unsqueeze(0).to(device)
move_tensor = torch.FloatTensor(move_features).unsqueeze(0).to(device)

# Forward pass
with torch.no_grad():
    policy_logits, value = model(board_tensor, rack_tensor, move_tensor)

print(f"✅ Forward pass successful!")
print(f"📊 Policy logits shape: {policy_logits.shape}")
print(f"📊 Value shape: {value.shape}")
print(f"📊 Value range: [{value.min().item():.3f}, {value.max().item():.3f}]")
